# Testing of the Deep CCA method

### Importing necessary packages:

In [6]:
import torch
import torch.nn as nn
import numpy as np
from DeepCCA.linear_cca import linear_cca
from torch.utils.data import BatchSampler, SequentialSampler
from DeepCCA.DeepCCAModels import DeepCCA
from DeepCCA.main import Solver
from DeepCCA.utils import load_data, svm_classify
try:
    import cPickle as thepickle
except ImportError:
    import _pickle as thepickle

import gzip

torch.set_default_tensor_type(torch.DoubleTensor)

### Parameters: